Créer une vue temporaire

In [9]:
%%sql
-- Créer une vue temporaire de la table Bronze
CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power AS
SELECT *
FROM Arailym_WindPowerAnalytics.LH_Wind_Power_Bronze.dbo.wind_power;

StatementMeta(, 15ca8c6a-8dcb-4352-adbc-5488d18fcdee, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

Appliquer les transformations SQL

In [10]:
%%sql
-- Nettoyer et enrichir les données
CREATE OR REPLACE TEMPORARY VIEW transformed_wind_power AS
SELECT
    production_id,
    date,
    turbine_name,
    capacity,
    location_name,
    latitude,
    longitude,
    region,
    status,
    responsible_department,
    wind_direction,
    
    -- 🔢 Arrondi des valeurs numériques
    ROUND(wind_speed, 2) AS wind_speed,
    ROUND(energy_produced, 2) AS energy_produced,
    
    -- 📅 Extraction des composants de date
    DAY(date) AS day,
    MONTH(date) AS month,
    QUARTER(date) AS quarter,
    YEAR(date) AS year,
    
    -- 🕐 Correction du format de time
    REGEXP_REPLACE(time, '-', ':') AS time,
    
    -- ⏰ Extraction des composants de temps
    CAST(SUBSTRING(time, 1, 2) AS INT) AS hour_of_day,
    CAST(SUBSTRING(time, 4, 2) AS INT) AS minute_of_hour,
    CAST(SUBSTRING(time, 7, 2) AS INT) AS second_of_minute,
    
    -- 🌅 Calcul de la période de la journée
    CASE
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 17 AND 20 THEN 'Evening'
        ELSE 'Night'
    END AS time_period
    
FROM bronze_wind_power;

StatementMeta(, 15ca8c6a-8dcb-4352-adbc-5488d18fcdee, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

Supprimer l'ancienne table Silver

In [11]:
%%sql
-- Supprimer l'ancienne table Silver si elle existe
DROP TABLE IF EXISTS WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power;

StatementMeta(, 15ca8c6a-8dcb-4352-adbc-5488d18fcdee, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

Créer la nouvelle table Silver


In [12]:
%%sql
-- Créer la nouvelle table Silver avec les données transformées
CREATE TABLE Arailym_WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power
USING delta
AS
SELECT * FROM transformed_wind_power;

StatementMeta(, 15ca8c6a-8dcb-4352-adbc-5488d18fcdee, 13, Finished, Available, Finished)

Error: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `chimcobldhq2agbic5kmoubdbtbmirj4a1nnepbi85n62r3pehkm6sp59h45ulr9dpi5uk3fetin4nqjd5m7cpbi4li64ro`.`wind_power` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

Vérification

In [13]:
%%sql
-- Vérifier que la table a été créée avec succès
SELECT 
    COUNT(*) as total_rows,
    MIN(date) as min_date,
    MAX(date) as max_date,
    COUNT(DISTINCT turbine_name) as turbine_count
FROM Arailym_WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power;

StatementMeta(, 15ca8c6a-8dcb-4352-adbc-5488d18fcdee, 14, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>